# Подготовка данных

Цель задания: определить уровень воды рек на постах гидрологического контроля, используя данные метеосводок и ежедневных наблюдений за 2008-2017 года.

### Проверка и установка рабочей директории, должен быть корень проекта

In [1]:
%pwd

'C:\\Users\\Kuroha\\source\\repos_py\\bauman_final_project\\notebooks'

In [2]:
%cd ..

C:\Users\Kuroha\source\repos_py\bauman_final_project


### Загрузка датасетов:

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from src.utils import *

KeyboardInterrupt: 

In [ ]:
def open_dataset(dataset_name):
    path = get_filepath(dataset_name, is_raw=True)
    return pd.read_csv(path, index_col=['uid', 'date'], parse_dates=['date'])

weather_df = open_dataset(DATA_WEATHER)
water_lvl_df = open_dataset(DATA_WATER_LEVEL)

### Информация о датасетах:

#### water_level

В датасете представлены замеры уровня воды для постов гидрологического контроля с сайта АИС ГМВО.

In [ ]:
water_lvl_df.head(3)

In [ ]:
water_lvl_df.info()

In [ ]:
water_lvl_df.columns

In [ ]:
water_lvl_df.shape

In [ ]:
water_lvl_df.describe().T

#### weather

В **weather** содержится погода на период 2008-2017 для обучения моделей, в котором есть следующие столбцы:
- индекс **uid** - идентификационный номер поста гидрологического контроля с сайта АИС ГМВО.
- индекс **date** - дата замера
- **temperature** - температура
- **cloud** - облачность
- **weather** - погодное явление

In [ ]:
weather_df.head(3)

In [ ]:
weather_df.info()

In [ ]:
weather_df.columns

In [ ]:
weather_df.shape

In [ ]:
weather_df.describe().T

***
### Объединение тренировочных наборов данных:

In [ ]:
df = weather_df.join(water_lvl_df)
df.head(), df.info(), df.shape, weather_df.shape

Количество строк до объединения **weather_df** и после осталось тем же.

### Работа с пропусками:

In [ ]:
print(f'Размерность water_lvl_df: {water_lvl_df.shape}')
print(f'Размерность weather_df: {weather_df.shape}')
print(f'Размерность df: {df.shape}')

В датасете **weather_df** есть строки за каждый день по каждому посту, однако в данных есть пропуски:

In [ ]:
df[df.isnull().any(axis=1)]

Рассмотрим данные с поста **9518** за период с **2016-09-17** по **2016-09-21**, где отсутствуют метео-данные:

In [ ]:
test_start_date = '2016-09-17'
test_end_date   = '2016-09-21'
df.query('uid == 9518 and date >= @test_start_date and date <= @test_end_date')

Посмотрим, в скольких строках отсутствуют данные:

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
print(f'Процент строк с отсутствующими данными об облачности: {df["cloud"].isnull().sum() / df.shape[0] * 100:.2f}%')
print(f'Процент строк с отсутствующими данными об уровне воды: {df["water_level"].isnull().sum() / df.shape[0] * 100:.2f}%')

Т.к. строк с частичными данными меньше 5%, то удалим их:

In [ ]:
df = df.dropna()
df.shape

In [ ]:
df.isnull().sum()

Чтобы в дальнейшем работать с текущими значениями внутри мультииндекса (uid и дата замера), создадим столбец с новым индексом:

In [ ]:
df.reset_index(inplace = True)
df.info()

### Обработка признаков:

#### Latitude (широта) и Longitude (долгота):

In [ ]:
def print_col_info(column):
    print(f'''describe:
{column.describe(datetime_is_numeric=True)}
{"-" * 80}
unique:
{column.unique()}
{"-" * 80}
nunique:
{column.nunique()}
{"-" * 80}
value_count:
{column.value_counts()}''')

print_col_info(df['latitude'])

In [ ]:
print_col_info(df['longitude'])

Всего уникальных локаций - 24.

In [ ]:
range_lat = range(int(df['latitude'].min()), int(df['latitude'].max() + 2))
range_long = range(int(df['longitude'].min()), int(df['longitude'].max() + 2))

ax = sns.scatterplot(data=df, x='latitude', y='longitude')
ax.set(xlabel='Долгота', ylabel='Широта', title='Локации постов наблюдений')
ax.set_xticks(range_lat)
ax.set_yticks(range_long)
plt.grid()
plt.show()

In [ ]:
print(f"Разброс долготы: {df['longitude'].min()} {df['longitude'].max()}")
print(f"Разброс широты: {df['latitude'].min()} {df['latitude'].max()}")

In [ ]:
print('Ссылка на Яндекс.Карты с выделенным регионом: ' +
f"https://yandex.ru/maps/?ll={(df['longitude'].max() + df['longitude'].min()) / 2}," +
f"{(df['latitude'].max() + df['latitude'].min()) / 2}" +  # начальные координаты для показа, покажем центр
f"&rl={df['longitude'].min()},{df['latitude'].min()}" +  # координаты первой точки выделения
f"~{df['longitude'].max() - df['longitude'].min()},0" +  # вторая точка, в виде смещения относительно начальной
f"~0,{df['latitude'].max() - df['latitude'].min()}" +
f"~{df['longitude'].min() - df['longitude'].max()},0" +
f"~0,{df['latitude'].min() - df['latitude'].max()}" +
f"&z=5")  # приближение на карте

![Регион на Яндекс.Картах](images/yandex_map.png)

Данные признаки будут нормализованы.

#### Cloud (облачность):

In [ ]:
print_col_info(df['cloud'])

Облачность может быть следующей:
- **sun** - ясно
- **sunс** - малооблачно
- **suncl** - облачно
- **dull** - пасмурно

Здесь прослеживается порядок - от ясного неба к пасмурному, поэтому для кодирования данного упорядоченного признака необходимо использовать метод Label Encoder.

Реализация данного метода в sklearn перед кодированием [сортирует уникальные признаки в алфавитном порядке](https://github.com/scikit-learn/scikit-learn/blob/f3f51f9b611bf873bd5836748647221480071a87/sklearn/preprocessing/_label.py#L799), в результате чего будет нарушен порядок: **dull** будет закодирован как 0, **sun** - как 1, **sunс** - 2, **suncl** - 3.

In [ ]:
df['cloud'] = df['cloud'].map({'sun': 0, 'sunc': 1, 'suncl': 2, 'dull': 3})
df.head(5)

#### uid:

In [ ]:
print_col_info(df['uid'])

Представляет собой идентификационный номер поста гидрологического контроля в базе данных сайта АИС ГМВО.

Задачу прогнозирования можно решить двумя способами:
1. Разработать одну модель для всех постов. Это имеет смысл, т.к. посты географически расположены близко друг к другу, а также замеряют уровень воды одной реки.
2. Разработать индивидуальные модели для всех постов.
    
Будет реализован первый вариант, т.к. недостаточно данных наблюдений по каждому посту. UID постов будут закодированы как категориальные данные, используя One Hot Encoding, однако предварительно нужно посмотреть, были ли случаи использования метео-данных по запасной локации - если их не было, то данные UIDы кодировать не нужно, т.к. их однозначно можно определить по координатам:

In [ ]:
ignore_uids = df.groupby('uid').filter(lambda x: (x.is_fallback_data == 0).all())['uid'].unique()
ignore_uids

In [ ]:
replace_map = dict(zip(ignore_uids, (None for x in range(len(ignore_uids)))))
replace_map

In [ ]:
df['uid_copy'] = df['uid']  # копия uid для визуализации, будет удалена перед сохранением датасета
df['uid_copy'] = df['uid_copy'].replace(replace_map)
print_col_info(df['uid_copy'])

In [ ]:
def encode_uid(df_target):
    encoder = OneHotEncoder()
    df_uid = pd.DataFrame(encoder.fit_transform(df[['uid_copy']]).toarray())
    df_uid = df_uid.add_prefix('uid_')  # префикс для визуального определения признака
    df_uid = df_uid.astype('category')  # конвертация в категориальный тип данных
    return df_target.join(df_uid)

df = encode_uid(df)
df.head(2)

In [ ]:
df.info()

In [ ]:
df.query('uid_copy.isnull()').head(1).T

None закодировался в uid_15, удалим столбцы uid_copy и uid_15:

In [ ]:
df = df.drop(['uid_copy', 'uid_15'], axis=1)
df.head(1).T

#### Date (дата):

In [ ]:
print_col_info(df['date'])

Представляет собой день наблюдений. Данное значение можно закодировать как:
1. Год - категориальный признак, используя метод Label Encoder (есть порядок: 2008 год был раньше, чем 2017)
2. Номер дня в году - цикличный признак.

Значения дня представляет собой следующий график:

In [ ]:
plt.figure(figsize=(15, 4))
plt.plot(df['date'].dt.day[:500])
plt.xlabel('Номер значения')
plt.ylabel('День')
plt.title('Значения дня')

plt.show()

Данные являются зацикленными, т.к. 31 день в месяце отличается от следующего 1-го дня лишь на одну единицу, а не на 30. Значение дня в году [можно представить в виде двух функций](http://blog.davidkaleko.com/feature-engineering-cyclical-features.html):

$x_{sin} = \sin(\frac{2 * \pi * x}{\max(x)})$

$x_{cos} = \cos(\frac{2 * \pi * x}{\max(x)})$

In [ ]:
test_day = 70
np.sin(2 * np.pi * test_day/365.0), np.cos(2 * np.pi * test_day/365.0)

In [ ]:
test_df = df[0:500]['date']
total_years = np.where(test_df.dt.is_leap_year, 366, 365)
test_arr = test_df.dt.dayofyear
test_sin = np.sin(2 * np.pi * test_arr / total_years)
test_cos = np.cos(2 * np.pi * test_arr / total_years)

fig = plt.figure(figsize=(15, 4))

ax1 = fig.add_subplot(121)
ax1.plot(test_sin, color='blue', label='sin')
ax1.plot(test_cos, color='red', label='cos')
ax1.legend()


# показ точек на графике за 300 дней

ax2 = fig.add_subplot(122)
ax2.set_aspect('equal')
ax2.scatter(test_sin[:300], test_cos[:300])
ax2.set_xlabel('sin')
ax2.set_ylabel('cos')

plt.show()

Закодируем год и номер дня в году:

In [ ]:
total_years = np.where(df['date'].dt.is_leap_year, 366, 365)
df['year'] = df['date'].dt.year
df['day_sin'] = np.sin(2 * np.pi * df['date'].dt.dayofyear / total_years)
df['day_cos'] = np.cos(2 * np.pi * df['date'].dt.dayofyear / total_years)
df[['date', 'year', 'day_sin', 'day_cos']].head()

In [ ]:
df['year'].value_counts().sort_index()

#### Weather (осадки)

In [ ]:
print_col_info(df['weather'])

Под осадками может пониматься следующее:
- **clear** - осадков не было
- **rain** - дождь
- **storm** - гроза
- **snow** - снег

Данный признак можно закодировать разными способами:
1. Выделение признака **наличие осадков**: и дождь, и снег образовываются из капель воды, а грозы, как правило, сопровождаются сильным дождём;
2. Объединение понятий "гроза" и "дождь", выделив 2 признака: **дождь** и **снег**;
3. 3 признака: **дождь**, **гроза**, **снег**, т.к. бывают сухие грозы;
4. В одном столбце будет указаны все осадки.

Во всех случаях отсутствие осадков обозначается 0 во всех признаках.

In [ ]:
df['weather_v1_precip'] = df['weather'].map({'clear': 0, 'rain': 1, 'storm': 1, 'snow': 1})

df['weather_v2_rain'] = df['weather'].map({'clear': 0, 'rain': 1, 'storm': 1, 'snow': 0})

# снег одинаково обозначается во 2 и 3 случаях
df['weather_snow'] = df['weather'].map({'clear': 0, 'rain': 0, 'storm': 0, 'snow': 1})

df['weather_v3_rain'] = df['weather'].map({'clear': 0, 'rain': 1, 'storm': 0, 'snow': 0})
df['weather_v3_storm'] = df['weather'].map({'clear': 0, 'rain': 0, 'storm': 1, 'snow': 0})

df['weather_v4'] = df['weather'].map({'clear': 0, 'rain': 1, 'storm': 2, 'snow': 3})

df = df.drop(['weather'], axis=1)

In [ ]:
df.head().T

### Визуализация статистики:

Динамика изменения уровня воды на примере 6 постов наблюдений:

In [ ]:
test_df = df[df['date'].dt.year == START_YEAR]

palette = sns.color_palette("husl", 28)
sns.relplot(data=test_df, x='date', y='water_level', palette=palette,
            hue='uid', kind="line", aspect=2, legend='full').set(
                title=f'Данные об уровнях воды всех постов за {START_YEAR} год')

In [ ]:
test_df = df[df['date'].dt.year == END_YEAR]

palette = sns.color_palette("husl", 28)
sns.relplot(data=test_df, x='date', y='water_level', palette=palette,
            hue='uid', kind="line", aspect=2, legend='full').set(
                title=f'Данные об уровнях воды всех постов за {END_YEAR} год')

Не для всех постов есть данные за каждый день, например:

In [ ]:
test_start_date = '2008-01-01'
test_end_date = '2008-12-31'

test_df = df.query('uid == 9421 and date >= @START_YEAR and date <= @test_end_date')
test_df['date'].head(3), test_df['date'].tail(3)

В 2008 году у поста 9421 (руч.без названия - факт.Кербо) есть показания от 1 мая до 25 ноября.

### Работа с аномалиями данных

In [ ]:
df.info()

Рассмотрим зависимости признаков:

In [ ]:
sns.pairplot(df[['water_level', 'temperature', 'cloud', 'weather_v1_precip']], diag_kind='kde')

По графикам видно, что уровень воды снижается при температуре ниже 0 градуса.

In [ ]:
fig = plt.figure(figsize=(15, 9))

ax1 = fig.add_subplot(221)
sns.boxplot(data=df, x='latitude', ax=ax1).set(title='Широта')

ax2 = fig.add_subplot(222)
sns.boxplot(data=df, x='longitude', ax=ax2).set(title='Долгота')

ax3 = fig.add_subplot(223)
sns.histplot(data=df, x='latitude', ax=ax3).set(title='Широта')

ax4 = fig.add_subplot(224)
sns.histplot(data=df, x='longitude', ax=ax4).set(title='Долгота')

plt.show()

Широта и долгота не имеет выбросов.

In [ ]:
heatmap = sns.heatmap(df.corr(), annot=True, linewidths=.5, fmt=".2f", robust=True)
heatmap.set_xticklabels(heatmap.get_xticklabels(), rotation = 30)

Температура значительно отрицательно коррелирует с day_cos.

## Нормализация данных

In [ ]:
df.select_dtypes(include=[np.number]).head(1)

In [ ]:
df['cloud'].value_counts()

Перед обучением модели необходимо привести нормализацию данных, а именно **temperature** и **year** (последний нужно закодировать с запасом на будущее).

In [ ]:
orig_first_year = df.at[0, 'year']
orig_first_year

Изменяем год в первом записи для того, чтобы нормализация по году прошла с учётом будущих годов:

In [ ]:
df['year'].head(2)

In [ ]:
df.at[0, 'year'] = 2030
df.at[0, 'year']

Перед нормализацией данных необходимо сохранить минимальные и максимальные значения:

In [ ]:
uid_min = df['uid'].min()
uid_max = df['uid'].max()
temperature_min = df['temperature'].min()
temperature_max = df['temperature'].max()
latitude_min = df['latitude'].min()
latitude_max = df['latitude'].max()
longitude_min = df['latitude'].min()
longitude_max = df['latitude'].max()

In [ ]:
from sklearn.preprocessing import MinMaxScaler, minmax_scale

columns_to_scale = ['temperature', 'cloud', 'year', 'latitude', 'longitude', 'uid']

df[columns_to_scale] = minmax_scale(df[columns_to_scale])
df[columns_to_scale]

Возвращаем обратно год в первой записи:

In [ ]:
df.at[0, 'year'] = df.at[1, 'year']
df.at[0, 'year']

## Сохранение данных

In [ ]:
df.to_csv(get_filepath(DATA_PROCESSED_TRAIN, is_raw=False), index=False)

Также сохраним пороговые значения для давления и скорости ветра для дальнейшего их применения к целевым данным, по которым будут производиться предсказания в дальнейшем:

In [ ]:
norm_info = {
    'year_first': START_YEAR,
    'year_last': 2030,
    'uid_min': uid_min,
    'uid_max': uid_max,
    'temperature_min': temperature_min,
    'temperature_max': temperature_max,
    'latitude_min': latitude_min,
    'latitude_max': latitude_max,
    'longitude_min': longitude_min,
    'longitude_max': longitude_max,
}
write_data(DATA_NORMALIZATION, data=norm_info, is_raw=True)